In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 15, Finished, Available, Finished)

In [ ]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 83feb432-fa84-49f0-9625-e626e739a0af)

In [ ]:
import synapse.ml.core  # Required Synapse ML Library
from synapse.ml.services import AnalyzeText  #Required pre-trained ML Model
# The Model AnalyzeText would be used to predict the sentiment of the news article.

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 17, Finished, Available, Finished)

In [ ]:
# We will be using the description column to predict the sentiment
# Importing the model and configuring the input and outout column below.

model = ( AnalyzeText()
        .setTextCol("description")      # Specify the column for the snetiment prediction. 
        .setKind("SentimentAnalysis")   # Specify the kind of analysis to be performed. AnalyzeTExt model can be used for performing multiple types of text analysis machine leanrning task like "language detector"
        .setOutputCol("response")       # Specify the column to store the output the model generates.   
        .setErrorCol("error"))          # Specify the colum which will hold the error value if anything goes wrong. If everything is right, it will have "None".




StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 18, Finished, Available, Finished)

In [ ]:
#Apply the model to out dataframe

result = model.transform(df) #"df" passed as input to the model.

# The model then takes the description column from the dataframe as input and predicts the sentiment and store the result in "response" column 
# and this putput will be store in another dataframe named "result". 


StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 19, Finished, Available, Finished)

In [ ]:
display(result)


StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8c813f42-6e95-499f-88ce-a99b54567329)

In [ ]:
# Create sentiment Column
from pyspark.sql.functions import col  # col function is used to refer a column in the dataframe

sentiment_df  = result.withColumn("sentiment", col("response.documents.sentiment"))  # sentiment column holds the sentiment extracted from response column.

display(sentiment_df)



StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8962a404-5e3d-4373-b16b-386122f61986)

In [ ]:
#We will be dropping the error and response column now. Sentiment has been retrieved
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 22, Finished, Available, Finished)

In [ ]:
display(sentiment_df_final)

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9a1c4436-14ee-4550-8936-41b0cc53b9e7)

In [ ]:
# To convert the datePublished column to date type before the merge operation since the table has date type column,.
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MM-yyyy"))


StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 24, Finished, Available, Finished)

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f""" 
                MERGE INTO {table_name} target_table
                USING vw_sentiment_df_final source_view

                ON source_view.url = target_table.url

                WHEN MATCHED AND
                source_view.title <> target_table.title  OR
                source_view.description <> target_table.description  OR
                source_view.category <> target_table.category  OR
                source_view.image <> target_table.image  OR
                source_view.provider <> target_table.provider  OR
                source_view.datePublished <> target_table.datePublished

                THEN UPDATE SET *

                WHEN NOT MATCHED THEN INSERT *    
    
    """)

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 25, Finished, Available, Finished)

Table Already Exists


T

In [ ]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_sentiment_analysis LIMIT 1000")
display(df)

StatementMeta(, acdb91f7-cb6a-4e03-8aa6-f464c786ab4d, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ce2aee5f-e5bc-4d4b-a35f-a3dc21aa903a)